In [1]:
import os, warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
warnings.filterwarnings('ignore') 

In [2]:
from random import shuffle
import sys, os
from datetime import datetime, timedelta
import numpy as np , pandas as pd
import time
import joblib
import random
import multiprocessing


from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args
from skopt import Optimizer # for the optimization
from joblib import Parallel, delayed # for the parallelization
import scipy


import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from vae_dense_model import VariationalAutoencoderDense as VAE_Dense
from vae_conv_model import VariationalAutoencoderConv as VAE_Conv
from vae_conv_I_model import VariationalAutoencoderConvInterpretable as VAE_ConvI
from config import config as cfg
import utils

# Define paths

In [3]:
input_dir = "../../data/processed_orig_data/"
output_dir = "../../data/generated_data/"
model_dir = './model/'
log_dir = './log/'
hpo_dir = './hpo_results/'

In [4]:
sys.path.insert(0, '../../evaluations/disc_and_preds/metrics/')

from discriminative_metrics3 import discriminative_score_metrics
from predictive_metrics3 import predictive_score_metrics
from visualization_metrics import visualization

# Utility Functions

In [5]:
# Set seed for reproducibility
def set_seeds(seed_value):   
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

In [6]:
def get_train_valid_split(data, valid_perc):
    N = data.shape[0]
    N_train = int(N * (1 - valid_perc))
    N_valid = N - N_train

    # shuffle data, just in case
    np.random.shuffle(data)

    # train, valid split 
    train_data = data[:N_train]
    valid_data = data[N_train:]
    return train_data, valid_data

In [7]:
def get_scaler(scaling_method): 
    scaler = None
    if scaling_method == 'minmax':    
        scaler = utils.MinMaxScaler( )  
    elif scaling_method == 'standard': 
        raise NotImplementedError(f'Scaling method {scaling_method} not implemented')
    elif scaling_method == 'yeojohnson':
        raise NotImplementedError(f'Scaling method {scaling_method} not implemented')
    else:         
        raise NotImplementedError(f'Scaling method {scaling_method} not implemented')  
        
    return scaler

In [8]:
def scale_train_valid_data(train_data, valid_data, scaling_method):         
         
          
    scaled_train_data = scaler.fit_transform(train_data)
    scaled_valid_data = scaler.transform(valid_data)
    return scaled_train_data, scaled_valid_data, scaler

## Main VAE Train and Evaluate Function

In [9]:
def train_model(train_data, latent_dim, n_layer1_in_25s, n_layer2_in_25s, n_layer3_in_50s, 
                reconstruction_wt, epochs = 100):
    
    _, T, D = train_data.shape

    # ----------------------------------------------------------------------------------------------
    # Instantiate the VAE
    vae = VAE_ConvI( seq_len=T,  
                    feat_dim = D, 
                    latent_dim = int(latent_dim), 
                    hidden_layer_sizes=[ 
                        int(n_layer1_in_25s*25), 
                        int(n_layer2_in_25s*25),
                        int(n_layer3_in_50s*50)], 
                    reconstruction_wt = reconstruction_wt, 
                # trend_poly=1, 
                # num_gen_seas=1,
                # custom_seas = [ (7, 1)] ,     # list of tuples of (num_of_seasons, len_per_season)
                use_residual_conn = True
        )

    vae.compile(optimizer=Adam())
    # vae.summary() ; sys.exit()

    # ----------------------------------------------------------------------------------------------
    # Train the VAE
    early_stop_loss = 'loss'
    early_stop_callback = EarlyStopping(monitor=early_stop_loss, min_delta = 1e-1, patience=50) 
    reduceLR = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10)

    history = vae.fit(
        train_data, 
        batch_size = 32,
        epochs=epochs,
        shuffle = True,
        callbacks=[early_stop_callback, reduceLR],
        verbose = 0
    )
    
    # ----------------------------------------------------------------------------------------------
    return vae, history

In [10]:
def generate_and_save_samples(vae, num_samples, scaler):
    samples = vae.get_prior_samples(num_samples= num_samples)
#     print("gen sample size: ", samples.shape)

    # inverse transform using scaler 
    samples = scaler.inverse_transform(samples)        

    # save to output dir
    samples_fpath = f'{model}/{model}_gen_samples_{data_name}_perc_{p}.npz'        
    np.savez_compressed(os.path.join( output_dir, samples_fpath), data=samples)


In [11]:
def evaluate_model(model, valid_data): 
    return model.evaluate(valid_data, verbose = 0, return_dict=True)

In [12]:
def confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

In [13]:
def evaluate_samples(scaled_ori_data, scaled_gen_data, predictor = 'conv', pred_epochs = 500, disc_epochs = 500):
    
    #orig_data, generated_data, epochs = 2500, predictor = 'conv'
    
    predictive_score = list(); discriminative_score = list()
    for tt in range(metric_iteration):
        pred_score = predictive_score_metrics(scaled_ori_data, scaled_gen_data, 
                                             predictor = 'conv', # conv, rnn, nbeats
                                             epochs = pred_epochs, )        
        
        predictive_score.append(pred_score)
        
#         disc_score = discriminative_score_metrics(scaled_ori_data, scaled_gen_data,  epochs = disc_epochs)
        disc_score = -1
        discriminative_score.append(disc_score)
        
#         print("iter: ", tt, 'pred_score', pred_score) 
        
    pred_mean = np.round(np.mean(predictive_score), 4)
    pred_CI = np.round(confidence_interval(predictive_score)[1], 4)
    
    disc_mean = np.round(np.mean(discriminative_score), 4)
    disc_CI = np.round(confidence_interval(discriminative_score)[1], 4)
    
    print('Predictive score: ', pred_mean, "+/-", pred_CI)  
    print('Discriminative score: ', disc_mean, "+/-", disc_CI)  
    
    return pred_mean, disc_mean

In [14]:
def evaluate_samples2(scaled_ori_data, scaled_gen_data, predictor = 'conv', pred_epochs = 500, disc_epochs = 500):
    
    pred_score = predictive_score_metrics(scaled_ori_data, scaled_gen_data, 
                                         predictor = 'conv', # conv, rnn, nbeats
                                         epochs = pred_epochs, )      

    # disc_score = discriminative_score_metrics(scaled_ori_data, scaled_gen_data,  epochs = disc_epochs)
    disc_score = -1    
    return pred_score, disc_score

# HPO Using Scikit Optimize

## Hyperparameter Space

In [100]:
# determine the hyperparameter space
param_grid = [
    Integer(2, 10, name="latent_dim"),    #1, 10
    
#     Integer(2, 3, name="n_layer1_in_25s"),  #1, 5
#     Integer(3, 4, name="n_layer2_in_25s"),  #1, 8
#     Integer(3, 4, name="n_layer3_in_50s"),  #1, 8
    Categorical([2], name="n_layer1_in_25s"),
    Categorical([2], name="n_layer2_in_25s"),
    Categorical([4], name="n_layer3_in_50s"),
    
    Categorical(['minmax'], name="scaling_method"),
    Real(0.5, 4.5, prior='uniform', name='reconstruction_wt'),
]

dim_names = [
    'latent_dim',
    'n_layer1_in_25s',
    'n_layer2_in_25s',
    'n_layer3_in_50s',
    'scaling_method',
    'reconstruction_wt',
]

default_parameters = [
    8,              # latent_dim
    2,              # n_layer1_in_25s 
    2,              # n_layer2_in_25s
    4,              # n_layer3_in_50s
    'minmax',        # scaling_method
    1.5,        # reconstruction_wt
]



default_parameters

[8, 2, 2, 4, 'minmax', 1.5]

## Objective for HPO

In [103]:
@use_named_args(param_grid)
def objective(
            latent_dim,
            n_layer1_in_25s,
            n_layer2_in_25s,
            n_layer3_in_50s,
            scaling_method,
            reconstruction_wt,
        ):

    
    start = time.time()  
    
    ## scale orig 
    scaler = get_scaler(scaling_method)
    scaled_ori_data = scaler.fit_transform(ori_data)   
    
    
    print("Training model")
    # train model 
    model, history = train_model(scaled_ori_data, 
            latent_dim, int(n_layer1_in_25s), int(n_layer2_in_25s), int(n_layer3_in_50s), reconstruction_wt,
            epochs = 2000)

    # generate and save samples
    scaled_gen_data = model.get_prior_samples(num_samples= scaled_ori_data.shape[0])    
#     print('orig and gen shapes', scaled_ori_data.shape, scaled_gen_data.shape)
    
    
    pred_scores = list(); disc_scores = list()
    for tt in range(metric_iteration):

        # evaluate the generated samples
        print("Scoring model ", tt)
        pred_score, disc_score = evaluate_samples2(
            scaled_ori_data, scaled_gen_data, predictor = 'conv', pred_epochs = pred_epochs, disc_epochs = disc_epochs)

        print("Model Score", tt, pred_score, disc_score)
        pred_scores.append(pred_score); disc_scores.append(disc_score)
    
    del model, history

    
    trial_loss_mean = np.round(np.mean(pred_scores), 4)
    trial_loss_CI = np.round(confidence_interval(pred_scores)[1], 4)
    trial_loss_max = np.round(np.max(pred_scores), 4)
    
    # Print the hyper-parameters and loss   
    print('-------------------------------------------')    
    print(f'latent_dim: {latent_dim}')
    print(f'n_layer1_in_25s: {n_layer1_in_25s}')
    print(f'n_layer2_in_25s: {n_layer2_in_25s}')
    print(f'n_layer3_in_50s: {n_layer3_in_50s}')
    print(f'reconstruction_wt: {reconstruction_wt}')
    print(f'scaling_method: {scaling_method}')   
    print()   
    print("all losses:", pred_scores)
    print(f"trial vae loss: {trial_loss_mean} +/- {trial_loss_CI}")
    print(f"Trial run time: {np.round((time.time() - start)/60.0, 2)} minutes") 
    print('-------------------------------------------')   
    
    return trial_loss_mean


# Main Time VAE HPO Loop, by dataset

# Multi-threaded

In [102]:
num_cpus_to_use = max(multiprocessing.cpu_count() - 2, 1)
num_cpus_to_use = 6
print("num_cpus_to_use: ", num_cpus_to_use)

metric_iteration = 4

pred_epochs = 500; disc_epochs = 500 


# num of trials for Bayesian search: initial and total (including initial)
n_initial_points = max(5, num_cpus_to_use)
n_calls = 50

# our model name
model = 'vae_conv_I'

dataset_names = ['stocks', 'stocks2', 'air', 'sine', 'energy']
# percs = [2, 5, 10, 20, 100]


# to custom run specific data
dataset_names = ['stocks2']
percs = [ 20 ]


# set random gen seed for reproducibiity
set_seeds(42)

main_start_time = time.time()    

best_loss = 1e9
for p in percs:  
    for data_name in dataset_names:   
        print("-"*80)
        print(f"dataset: {data_name}, perc: {p}")
        # --------------------------------------------------------------------
        ### file name to load
        fname = f'{input_dir + data_name}_subsampled_train_perc_{p}.npz'
        
        ### read data        
        loaded = np.load(fname)
        ori_data = loaded['data']  
        N = ori_data.shape[0]
        # ori_data = ori_data[np.random.choice(N, N//2, replace=False), :]
        # print(ori_data.shape)     
        # --------------------------------------------------------------------
        
        optimizer = Optimizer(
            dimensions = param_grid, # the hyperparameter space
            base_estimator = "GP", # the surrogate
            n_initial_points=n_initial_points, # the number of points to evaluate f(x) to start of
            acq_func='EI', # the acquisition function
            random_state=0, 
            n_jobs=num_cpus_to_use,
        )
        # --------------------------------------------------------------------
        
        num_loops = int(np.ceil(n_calls / num_cpus_to_use))
        print(f'num_loops: {num_loops}')
        
        x_dict = {}
        
        for i in range(num_loops): 
            print('-'*80)
            x = []; 
            pts = optimizer.ask(n_points=50)
            for pt in pts: 
                pt_key = ''.join(str(s) for s in pt)
                if pt_key in x_dict: continue
                x_dict[pt_key] = 1
                x.append(pt)
                if len(x) == num_cpus_to_use: break

    
            y = Parallel(n_jobs=num_cpus_to_use)(delayed(objective)(v) for v in x)  # evaluate points in parallel
            optimizer.tell(x, y)
            print('\nloop:', i, 'points:', x, y)
            
            cumu_time = np.round((time.time() - main_start_time)/60.0, 2)
            print("iter:", i, 'vae loss: ', optimizer.yi[-1], 'cumu_time_mins:', cumu_time)
            
            #technially should be outside the for loop but saving periodically in case it fails 
            hpo_results = pd.concat([
                pd.DataFrame(optimizer.Xi),
                pd.Series(optimizer.yi),
            ], axis=1)

            hpo_results.columns = dim_names + ['loss']
            hpo_results.insert(0, 'dataset', data_name)
            hpo_results.insert(1, 'perc', p)

            file_name = f'hpo_results_{model}_{data_name}_perc_{p}_test.csv'   # 
            hpo_results.to_csv(os.path.join( hpo_dir, file_name), index=False)
        

end = time.time()
elapsed_time = np.round((end - main_start_time)/60.0, 2)
print(f"All done in {elapsed_time} minutes!")  

num_cpus_to_use:  6
--------------------------------------------------------------------------------
dataset: stocks2, perc: 20
num_loops: 9
--------------------------------------------------------------------------------


UnboundLocalError: local variable 'trial_num' referenced before assignment

# Inspect HPO Results 

### Load Data

In [96]:
test_data = 'stocks'
test_perc = 5

file_name = f'hpo_results_{model}_{test_data}_perc_{test_perc}_test.csv'   # 
hpt_results = pd.read_csv(os.path.join( hpo_dir, file_name))

# function value at the minimum.
print("Best score=%.4f" % hpt_results['loss'].min())

hpt_results.sort_values(by='loss', inplace=True, ascending=False)
hpt_results.reset_index(drop=True, inplace=True)
hpt_results.tail(10)

Best score=0.1110


,dataset,perc,latent_dim,n_layer1_in_25s,n_layer2_in_25s,n_layer3_in_50s,scaling_method,reconstruction_wt,loss
44,stocks,5,2,1,2,3,minmax,0.803957,0.1151
45,stocks,5,2,1,2,3,minmax,0.820629,0.1147
46,stocks,5,7,3,2,5,minmax,0.874990,0.1145
47,stocks,5,6,2,2,4,minmax,2.957224,0.1144
48,stocks,5,4,2,1,4,minmax,2.016872,0.1143
49,stocks,5,6,2,2,3,minmax,1.169333,0.1141
50,stocks,5,2,1,1,5,minmax,0.800000,0.1136
51,stocks,5,2,1,1,5,minmax,1.663378,0.1132
52,stocks,5,2,3,2,3,minmax,3.000000,0.1120
53,stocks,5,2,3,2,3,minmax,0.873437,0.1110


# Check Convergence 

In [ ]:
hpt_results['loss'].plot()